In [1]:
#Import the Dependencies
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [2]:
#check for CUDA or rocm availability
device = torch.device("rocm" if torch.cuda.is_available() else "cpu")
print(f"Using Device: {device}")

Using Device: cpu


Data Collection and preprocessing 

In [3]:
#load the data from library

data = load_breast_cancer()
x = data.data
y = data.target

In [4]:
print(x)

[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.871e-01 7.039e-02]]


In [5]:
#Spliting the data into training and testing set
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [6]:
#standardize the data using standard scalers

scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

For pytorch we have to convert the data to Pytorch_tensor basically pytorch dont work great with numpy


In [7]:
#Converting data to PyTorch tensors and move it to GPU/CPU
x_train = torch.tensor(x_train,dtype=torch.float32).to(device) #Here Device implese the resouce which we call for working if its GPU it will call for it else CPU will be used
x_test = torch.tensor(x_test,dtype=torch.float32).to(device)
y_train = torch.tensor(y_train,dtype=torch.float32).to(device)
y_test = torch.tensor(y_test,dtype=torch.float32).to(device)

Neural Network Architecture

In [8]:
#Define Neural Network Architecture
class NuralNet(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(NuralNet,self).__init__()
        self.fc1 = nn.Linear(input_size,hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size,output_size)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self,x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

In [9]:
#Define Hyperparameters

input_size = x_train.shape[1]
hidden_size = 128
output_size = 1
lerning_rate = 0.001
num_epochs = 100

In [10]:
#Ininalize the nural network and move it the GPU

model = NuralNet(input_size,hidden_size,output_size).to(device)

In [11]:
#Define loss and the optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(),lr=lerning_rate)

Training the Neural Network

In [12]:
#Training the Model
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(x_train)
    loss = criterion(outputs, y_train.view(-1,1))
    loss.backward()
    optimizer.step()


    #Calculate the accuracy
    with torch.no_grad():
        predicted = outputs.round()
        correct = (predicted == y_train.view(-1,1)).float().sum()
        accuracy = correct / y_train.size(0)


    if (epoch +1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss : {loss.item():.4f} , Accuracy: {accuracy.item()*100:.2f}%')

Epoch [10/100], Loss : 0.4993 , Accuracy: 92.31%
Epoch [20/100], Loss : 0.3554 , Accuracy: 94.29%
Epoch [30/100], Loss : 0.2619 , Accuracy: 94.95%
Epoch [40/100], Loss : 0.2022 , Accuracy: 95.60%
Epoch [50/100], Loss : 0.1646 , Accuracy: 95.82%
Epoch [60/100], Loss : 0.1399 , Accuracy: 96.70%
Epoch [70/100], Loss : 0.1225 , Accuracy: 97.14%
Epoch [80/100], Loss : 0.1094 , Accuracy: 97.80%
Epoch [90/100], Loss : 0.0991 , Accuracy: 97.80%
Epoch [100/100], Loss : 0.0908 , Accuracy: 97.58%


Model Evaluation 

In [13]:
# Evaluation on training set
model.eval()
with torch.no_grad():
    outputs = model(x_train)
    predicted = outputs.round()
    correct = (predicted == y_train.view(-1,1)).float().sum()
    accuracy = correct / y_train.size(0)
    print(f"Accuracy on Training Data: {accuracy.item()*100:.2f}%")

Accuracy on Training Data: 97.80%


In [14]:
# Evaluation on testing set
model.eval()
with torch.no_grad():
    outputs = model(x_test)
    predicted = outputs.round()
    correct = (predicted == y_test.view(-1,1)).float().sum()
    accuracy = correct / y_test.size(0)
    print(f"Accuracy on Testing Data: {accuracy.item()*100:.2f}%")

Accuracy on Testing Data: 97.37%


In [16]:
import pickle 

In [17]:
pickle.dump(model,open("Breast_Cancer_Prediction_PyTorch.sav",'wb') )